# 01 SIMD and data cubes

In [15]:
import sys
sys.path.append("../")
import wonky as w
import numpy as np
import pandas as pd
cube = np.load("../sample_data/exp/00.npy")
cube.shape
#spcies * observables * <x>^n * time intervals 

(1, 2, 9, 20)

In [16]:
species, observable = 0,1
#view a slice in a dataframe
pd.DataFrame(cube[species][observable]).T.head()

,0,1,2,3,4,5,6,7,8
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,2.0,4.0,8.0,16.0,32.0,64.0,128.0,256.0
2,1.0,3.0,9.0,27.0,81.0,243.0,729.0,2187.0,6561.0
3,1.0,4.0,16.0,64.0,256.0,1024.0,4096.0,16384.0,65536.0
4,1.0,4.0,16.0,64.0,256.0,1024.0,4096.0,16384.0,65536.0


with numba, it is meaningful to mix vector based and procedural code if we like - wonky stats module updates as follows

In [17]:
from numba import jit

#statistics would be constructed with a cube shape
pseudo_self = w.data.statistics((1,2,20))
#stats contains this function, pulled out for inspection
@jit
def update(self, t, v):
    self._counter += v
    self._last_t = t
    for o in range(self._num_obs):
        for s in range(self._num_species):
            for m in range(self._num_moments):   
                self[s][o][m][t] = self._counter[o][s]**m
                
pseudo_self

statistics([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
               0., 0., 0., 0., 0., 0.]],


In [18]:
#first time it is compiled JIT
update(pseudo_self, 1, [ [2], [1] ])
pseudo_self.astype(int)

statistics([[[[  0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0,   8,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0,  32,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0,  64,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0],
              [  0, 128,   0,   0,   0,   0,   0,   0,   0,   0,   0,
             

In [19]:
rsys =  w.reaction.reaction_system([w.reaction("A", "A+A+B", 0.5),
                                        w.reaction("A", "0", 0.5),
                                   w.reaction("A+B", "B", 0.5),
                                        w.reaction("B", "0", 0.5)])
rsys

(array([[[1, 2],
         [0, 1]],
 
        [[1, 0],
         [0, 0]],
 
        [[1, 0],
         [1, 1]],
 
        [[0, 0],
         [1, 0]]]), {A: 0, B: 1}, array([0.5, 0.5, 0.5, 0.5]))

In [20]:
#reactions * species * IO
sample_reaction = rsys[0][2]
rsys[0].shape

(4, 2, 2)

## Sample site and then apply creation/annihilation 

This is almost trivial, merely demonstrates that we can think about the shape of the tensors and construct algorithms in this way

In [21]:
site = np.array([1,1])
delta =  sample_reaction[:,1]-sample_reaction[:,0]
delta

array([-1,  0])

In [22]:
#the reaction reslts in a species vector delta which can be applied to the site
site += delta
site

array([0, 1])

# 03 Code and data interfaces

## Example code interface for simulations

Good

In [23]:
def put (lattice,i,species):
    lattice[i] |= (1<<species)
    return lattice[i]

bad 

In [24]:
def diffuse(i, length, dim):
    def __wrapped__(i, idash,length, axis,jump):
        boundary = jump * length
        if (np.floor(i/boundary)!=np.floor(idash/boundary)):  
            return int((idash + boundary) % boundary)
        return idash
    
    print("diffusing on a {}x{} lattice".format(length,dim))
    choice = np.random.randint(2*dim)
    axis = int(choice/2.)
    sense = -1 if choice % 2 == 0 else 1
    jump = length**axis
    return __wrapped__(i, i + (sense * jump),length,axis,jump*1.)

A contrived example of loading functions: using a closure to return functions that have the same interface

In [25]:
def diffuse_func_factory(graph_type, *args):
    if graph_type == 0:
        def __func__(i):
            return diffuse(i,*args)
        return __func__
    if graph_type == 1:
        #return another fuction e.g. somethings that loads an adjacency matrix for a scale free network
        def __func__(i):
            dummy_adj = np.array([[1,2,3,4],
                                  [5,6,7,8]])
            return np.random.choice(dummy_adj[i])
        return __func__

for a regular lattice I asked for type 0 with some params and then I hold onto the function

In [26]:
my_diffuser = diffuse_func_factory(0, 10,2)
#THIS THING IS A FUNCTION
my_diffuser

<function __main__.diffuse_func_factory.<locals>.__func__>

In [27]:
#use it by supplying only the site id, I do not need to supply dimensions each time
my_diffuser(10)

diffusing on a 10x2 lattice


11

In [28]:
#Or I can use a different graph with the same interface
my_diffuser = diffuse_func_factory(1)
my_diffuser(1)

7

## Example [data] interface

We already saw this with the example cube. Does not matter what the convention is but should be "dimensionally consistent" across models and experiments with some metadata to describe the dimensions

# 03 Dependency injection
Dep injection goes hand in hand with interfaces. For example on loading Wonky, we might use a config file to specify what types of graphs or lattices our diffusion factory should load or we might use conditional imports in python. 

This is different than putting switch statements in our code because we only "compile" the module we want and the code is truly invariant to choice of graph or whatever the thing is. Care required in practice.

Key point is to respect modularity, separation of concerns and clean interfaces; for numerical code, this should be employed only in scaffolding phases and not in sim loops! Obviously.